# Handling multiple sequences

Önceki bölümde, en basit kullanım durumlarını inceledik: küçük uzunluktaki tek bir dizi üzerinde çıkarım yapmak. Ancak, bazı sorular şimdiden ortaya çıkmaktadır:

- Birden fazla diziyi nasıl ele alacağız?
- Farklı uzunluklardaki çoklu dizileri nasıl ele alacağız?
- Bir modelin iyi çalışmasını sağlayan tek girdi kelime indeksleri midir?
- Çok uzun dizi diye bir şey var mıdır?

Bu soruların ne tür sorunlar ortaya çıkardığını ve Transformers API kullanarak bunları nasıl çözebileceğimizi görelim.

## Models expect a batch of inputs

Önceki alıştırmada dizilerin sayı listelerine nasıl çevrildiğini gördünüz. Bu sayı listesini bir tensöre dönüştürelim ve modele gönderelim:

In [1]:
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

# This line will fail.
model(input_ids)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

IndexError: too many indices for tensor of dimension 1

Olamaz! Bu neden başarısız oldu? "Bölüm 2'deki pipeline'daki adımları takip ettik.

Sorun şu ki, modele tek bir dizi gönderdik, oysa Transformers modelleri varsayılan olarak birden fazla cümle bekler. Burada, bir diziye uyguladığımızda tokenizer'ın sahne arkasında yaptığı her şeyi yapmaya çalıştık. Ancak yakından bakarsanız, tokenizer'ın yalnızca girdi kimlikleri listesini bir tensöre dönüştürmediğini, bunun üzerine bir boyut eklediğini göreceksiniz:

In [2]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


Tekrar deneyelim ve yeni bir boyut ekleyelim:

In [3]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Yığınlama, birden fazla cümleyi model üzerinden tek seferde gönderme eylemidir. Yalnızca bir cümleniz varsa, tek bir diziyle bir toplu iş oluşturabilirsiniz:

Batching, modeli birden fazla cümleyle beslediğinizde çalışmasını sağlar. Birden fazla dizi kullanmak, tek bir diziyle bir toplu iş oluşturmak kadar basittir. Yine de ikinci bir sorun var. İki (veya daha fazla) cümleyi bir araya getirmeye çalıştığınızda, bunlar farklı uzunluklarda olabilir. Daha önce tensörlerle çalıştıysanız, bunların dikdörtgen şeklinde olması gerektiğini bilirsiniz, bu nedenle giriş kimlikleri listesini doğrudan bir tensöre dönüştüremezsiniz. Bu sorunu aşmak için genellikle girdileri **padding** işlemi uygularız.

## Padding the inputs

Aşağıdaki listeler bir tensöre dönüştürülemez:

```
batched_ids = [
    [200, 200, 200],
    [200, 200]
]
```

Bunu aşmak için, tensörlerimizin dikdörtgen bir şekle sahip olmasını sağlamak için padding kullanacağız. Padding, daha az değere sahip cümlelere padding token adı verilen özel bir kelime ekleyerek tüm cümlelerimizin aynı uzunlukta olmasını sağlar. Örneğin, 10 kelimelik 10 cümleniz ve 20 kelimelik 1 cümleniz varsa, padding tüm cümlelerin 20 kelimeye sahip olmasını sağlayacaktır. Örneğimizde, ortaya çıkan tensör aşağıdaki gibi görünür:

```
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]
```

Dolgu belirteci kimliği `tokenizer.pad_token_id` içinde bulunabilir. Bunu kullanalım ve iki cümlemizi model üzerinden ayrı ayrı ve birlikte gönderelim:

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


Toplu tahminlerimizdeki logitlerde bir sorun var: ikinci satır, ikinci cümlenin logitleriyle aynı olmalıdır, ancak tamamen farklı değerler elde ettik!

Bunun nedeni, Transformer modellerinin temel özelliğinin her bir tokeni bağlamsallaştıran dikkat katmanları olmasıdır. Bunlar, bir dizideki tüm tokenlara katıldıkları için dolgu tokenlarını dikkate alacaktır. Farklı uzunluklardaki cümleleri tek tek modelden geçirirken ya da aynı cümleleri içeren ve dolgu uygulanmış bir yığını geçirirken aynı sonucu elde etmek için, bu dikkat katmanlarına dolgu belirteçlerini göz ardı etmelerini söylememiz gerekir. Bu, bir dikkat maskesi kullanılarak yapılır.

## Attention masks

Dikkat maskeleri, girdi kimlikleri tensörü ile tamamen aynı şekle sahip, 0 ve 1'lerle doldurulmuş tensörlerdir: 1'ler ilgili tokenlara dikkat edilmesi gerektiğini ve 0'lar ilgili tokenlara dikkat edilmemesi gerektiğini (yani, modelin dikkat katmanları tarafından göz ardı edilmeleri gerektiğini) gösterir.

Önceki örneği bir dikkat maskesi ile tamamlayalım:

In [5]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(
    torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask)
)
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


Şimdi de yığındaki ikinci cümle için aynı logitleri elde ediyoruz.

İkinci dizinin son değerinin dikkat maskesinde 0 değeri olan bir dolgu kimliği olduğuna dikkat edin.

## Longer sequences

Transformer modellerinde, modellere aktarabileceğimiz dizilerin uzunlukları için bir sınır vardır. Çoğu model 512 veya 1024 token'a kadar olan dizileri işler ve daha uzun dizileri işlemesi istendiğinde çökecektir. Bu sorunun iki çözümü vardır:

- Daha uzun desteklenen dizi uzunluğuna sahip bir model kullanın.
- Dizilerinizi kısaltın.

Modeller farklı desteklenen dizi uzunluklarına sahiptir ve bazıları çok uzun dizileri işleme konusunda uzmanlaşmıştır. Longformer buna bir örnektir ve bir diğeri de LED'dir. Çok uzun sekanslar gerektiren bir görev üzerinde çalışıyorsanız, bu modellere göz atmanızı öneririz.

Aksi takdirde, `max_sequence_length` parametresini belirterek dizilerinizi kesmenizi öneririz: